In [1]:
import findspark
findspark.init()

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3" pyspark-shell'

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Twitter")
    .getOrCreate())

Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
:: loading settings :: url = jar:file:/opt/spark3/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1517629e-f3c1-4e86-bd6f-1f17cabbd1b6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.3 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 2145ms :: artifacts dl 72ms
	:: modules in us

In [6]:
schema="""
created_at string,
id bigint,
id_str string,
text string,
source string,
truncated boolean,
in_reply_to_status_id bigint,
in_reply_to_status_id_str string,
in_reply_to_user_id bigint,
in_reply_to_user_id_str string,
in_reply_to_screen_name string,
`user` struct<
            id:bigint,
            id_str:string,
            name:string,
            screen_name:string,
            location:string,
            url:string,
            description:string,
            protected:boolean,
            verified:boolean,
            followers_count:bigint,
            friends_count:bigint,
            listed_count:bigint,
            favourites_count:bigint,
            statuses_count:bigint,
            created_at:string,
            profile_banner_url:string,
            profile_image_url_https:string,
            default_profile:boolean,
            default_profile_image:boolean,
            withheld_in_countries: array<string>,
            withheld_scope:string,
            geo_enabled:boolean
            >,
coordinates struct <
            coordinates:array<float>,
            type:string
            >,
place struct<
            country:string,
            country_code:string,
            full_name:string,
            place_type:string,
            url:string
            >,
quoted_status_id bigint,
quoted_status_id_str string,
is_quote_status boolean,
quote_count bigint,
reply_count bigint,
retweet_count bigint,
favorite_count bigint,
entities struct<
            user_mentions:array<struct<screen_name:string>>,
            hashtags:array<struct<text:string>>, 
            media:array<struct<expanded_url:string>>, 
            urls:array<struct<expanded_url:string>>, 
            symbols:array<struct<text:string>>
            >,
favorited boolean,
retweeted boolean,
possibly_sensitive boolean,
filter_level string,
lang string
"""

In [3]:
events = spark.readStream \
               .format("kafka") \
               .option("kafka.bootstrap.servers", "localhost:9092") \
               .option("subscribe", "tweets") \
               .option("startingOffsets", "latest") \
               .load()
               #.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"))


In [4]:
events.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
personStringdf = events.selectExpr("CAST(value AS STRING)")

In [6]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import from_json, col


schema = StructType() \
    .add("text", StringType()) \
    
personDF = personStringdf.select(from_json(col("value"), schema).alias("data")).select("data.*")

In [9]:
#Para visualizar el texto en el notebook

#personDF.writeStream \
      #.format("console") \
      #.outputMode("append") \
      #.start() \
      #.awaitTermination()


-------------------------------------------
Batch: 0
-------------------------------------------
+----+
|text|
+----+
+----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|@Safira138930442 ...|
|$250 💵💵💵

$250...|
|RT @YoHViral: Je ...|
|RT @albabrokers: ...|
|@Trader_XO I thin...|
|@Ksadajuguetona @...|
|#BTC :$38545.31
#...|
|RT @Chek_24h: I f...|
+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+----------------------+
|                  text|
+----------------------+
|  $Looks alan agala...|
|  @TheMoonCarl $CIS...|
|RT @gojogojou: 【仮...|
|  The words “curren...|
|  @OrijinHQ Love yo...|
|  RT @BitShiba: You...|
|  @DeadApeClub @pan...|
|  RT @Shido_samurai...|
+----------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @BitShiba: You...|
|RT @Scamco_Chairm...|
|RT @Tronmining: $...

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @Cryptoboss200...|
|RT @CaptainCrypto...|
|$BTC showing very...|
|RT @CaptainCrypto...|
|RT @Tronmining: $...|
+--------------------+

-------------------------------------------
Batch: 9
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @zabad_kassim:...|
|@ShibInfo #MetaRu...|
|RT @dWebGuide: If...|
+--------------------+



-------------------------------------------
Batch: 10
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @Chek_24h: I f...|
|RT @GpUeco: The f...|
+--------------------+

-------------------------------------------
Batch: 11
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @lawmaster: On...|
|RT @ferigle: 👀 S...|
|RT @TheEliteCrypt...|
+--------------------+

-------------------------------------------
Batch: 12
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @vipjujubrais:...|
|RT @NervChain: He...|
|RT @BitriseToken:...|
|RT @Tronmining: $...|
+--------------------+

-------------------------------------------
Batch: 13
-------------------------------------------
+------------------------------------+
|                                text|
+------------------------------------+
|     

-------------------------------------------
Batch: 14
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @ethbabesnft: ...|
+--------------------+

-------------------------------------------
Batch: 15
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|@ElonHusky_ @ccti...|
|Check out this it...|
|This project is l...|
|@ThinkingBitmex G...|
|RT @Revolotto: 🌍...|
|نعم يوجد على تريد...|
|@cobie Holding BT...|
|RT @ethbabesnft: ...|
+--------------------+

-------------------------------------------
Batch: 16
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|RT @PARS_ALFATRAD...|
+--------------------+

-------------------------------------------
Batch: 17
-------------------------------------------
+--------------------+
|                text|
+--------------------+
|BARUSDT 15m Price...|
+-------

KeyboardInterrupt: 

In [10]:
personDF.selectExpr("to_json(struct(*)) AS value") \
   .writeStream \
   .format("kafka") \
   .outputMode("append") \
   .option("kafka.bootstrap.servers", "localhost:9092") \
   .option("checkpointLocation", "/tmp/spark_checkpoint") \
   .option("topic", "alerts") \
   .start() \
   .awaitTermination()

KeyboardInterrupt: 